In [1]:
import pandas as pd 
import numpy as np
import matplotlib.pyplot as plt
import os

In [14]:
covid_data = pd.read_csv(os.path.join("..", "Datasets", "covid_iledefrance.csv"))
test_frame = pd.read_parquet(os.path.join("..", "Datasets", "train.parquet"))

In [15]:
test_frame_target = test_frame[['date','log_bike_count']]

In [31]:
test_frame_target['date'] = pd.to_datetime(test_frame_target['date'])

# Extract date without considering the time component
test_frame_target['day'] = test_frame_target['date'].dt.date

# Group by the 'day' and sum the 'log_bike_count'
daily_sum = test_frame_target.groupby('day')['log_bike_count'].sum().reset_index()
daily_sum.set_index('day', inplace=True)


C:\Users\35387\AppData\Local\Temp\ipykernel_15584\4274184649.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  test_frame_target['date'] = pd.to_datetime(test_frame_target['date'])
C:\Users\35387\AppData\Local\Temp\ipykernel_15584\4274184649.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  test_frame_target['day'] = test_frame_target['date'].dt.date


In [25]:
covid_data['date'] = pd.to_datetime(covid_data['date'])
covid_summed = covid_data.groupby('date').sum(numeric_only=True).reset_index()
covid_summed.drop(['dep','reg'], axis=1, inplace=True)  # These have become meaningless
covid_summed.set_index('date', inplace=True)

In [26]:
covid_summed.shape

(1200, 17)

In [27]:
covid_data.shape

(9600, 22)

In [33]:
merged_frame = pd.merge(daily_sum, covid_summed, left_index=True, right_index=True, how='inner')


In [36]:
column_of_interest = 'log_bike_count'

# Calculate the correlation matrix
correlation_matrix = merged_frame.corr()

# Display the correlations of the chosen column with all other columns
correlations_with_column = correlation_matrix[column_of_interest]

In [37]:
correlations_with_column

log_bike_count    1.000000
tx_pos           -0.301409
tx_incid         -0.272389
TO               -0.351145
R                -0.047457
hosp             -0.561063
rea              -0.351145
rad               0.280755
dchosp            0.256788
reg_rea          -0.351145
incid_hosp       -0.285313
incid_rea        -0.227526
incid_rad        -0.138725
incid_dchosp     -0.371013
reg_incid_rea    -0.227526
pos              -0.010996
pos_7j           -0.267003
cv_dose1               NaN
Name: log_bike_count, dtype: float64

In [ ]:
# ok we will fit on incid_dchosp

### Preparing to build the class